In [ ]:
!pip install category_encoders==2.*
#!pip install pandas_profiling==2.*

     |████████████████████████████████| 80 kB 4.7 MB/s 


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# pd.options.display.max_rows = 100

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Decision Trees

- clean data with **outliers and missing values**
- use scikit-learn for **decision trees**
- get and interpret **feature importances** of a tree-based model
- understand why decision trees are useful to model 

# I. Wrangle Data

### Import data



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.merge(pd.read_csv('train_features.csv', na_values = [0]), pd.read_csv('train_labels.csv', na_values = [0]))


FileNotFoundError: ignored

In [ ]:
def wrangle(temp_df):

  #set row index to 'id column
  temp_df.set_index('id', inplace = True)

  #Drop the constant column 'recorded by' and the duplicate column 'quantity group'
  df.drop(columns=['recorded_by', 'quantity_group'], inplace = True, errors='ignore')

  # Drop High-Cardinality Columns
  cols_to_drop = [col for col in df.select_dtypes('object') if df[col].nunique() > 100]
  df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
  
  # Drop columns with a high proportion of null values
  df.drop(columns='num_private', inplace=True, errors='ignore')
  
  return temp_df


In [ ]:
#df['recorded_by'].value_counts()
#df['num_private'].value_counts(dropna=False)

In [ ]:
train = wrangle(df)

### EDA

In [ ]:
train.shape
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
import seaborn as sns

sns.scatterplot(x = 'longitude', y = 'latitude', data = train, alpha = .01)

In [ ]:
sns.set(rc={'figure.figsize':(15, 10)})
sns.scatterplot(x = 'longitude', y = 'latitude', data = train,  hue = 'status_group')

In [ ]:
#report = ProfileReport(train, minimal=True).to_notebook_iframe()

#report

# II. Split Data

## Split Target Vector from Feature Matrix

In [ ]:
target = 'status_group'

y = train[target]

X = train.drop(columns = target)

## Training-Validation Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state = 5)

# III. Establish Baseline

- Is this a *regression* or a *classification* problem?
- Because it's a **classification** problem, we need to calculate baseline accuracy based on our majority class. 

In [ ]:
print('Baseline Accurary', y_train.value_counts(normalize = True).max()*100)

# IV. Build Model(s)

- Encoder for categorical features.
- Imputer for missing values.
- `StandardScaler` for `LogisticRegression`

**First Model:** Logistic Regression

In [ ]:
model_lr = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    LogisticRegression(max_iter=200)
)

model_lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['basin', 'region', 'public_meeting',
                                     'scheme_management', 'permit',
                                     'extraction_type', 'extraction_type_group',
                                     'extraction_type_class', 'management',
                                     'management_group', 'payment',
                                     'payment_type', 'water_quality',
                                     'quality_group', 'quantity', 'source',
                                     'source_type', 'source_class',
                                     'waterpoint_type',...
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    

**Second Model:** Decision Tree Classifier

In [ ]:
model_dt = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    DecisionTreeClassifier(random_state= 42, max_depth=16)
)

model_dt.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['basin', 'region', 'public_meeting',
                                     'scheme_management', 'permit',
                                     'extraction_type', 'extraction_type_group',
                                     'extraction_type_class', 'management',
                                     'management_group', 'payment',
                                     'payment_type', 'water_quality',
                                     'quality_group', 'quantity', 'source',
                                     'source_type', 'source_class',
                                     'waterpoint_type',...
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=16,
                                       

**Interlude: How does a tree model work?**

In [ ]:
plt.figure(figsize = (60,15))

plot_tree(model_dt.named_steps['decisiontreeclassifier'],
          max_depth = 4,
      #    feature_names = X_train.columns,
          filled = True)

[Text(1674.0, 747.4499999999999, 'X[141] <= 1.273\ngini = 0.552\nsamples = 38016\nvalue = [20676, 2740, 14600]'),
 Text(837.0, 611.55, 'X[168] <= 1.28\ngini = 0.528\nsamples = 33955\nvalue = [20569, 2711, 10675]'),
 Text(418.5, 475.65, 'X[167] <= 1.309\ngini = 0.499\nsamples = 30814\nvalue = [20035, 2529, 8250]'),
 Text(209.25, 339.74999999999994, 'X[112] <= 0.149\ngini = 0.474\nsamples = 27475\nvalue = [18619, 2101, 6755]'),
 Text(104.625, 203.8499999999999, 'X[15] <= 1.438\ngini = 0.413\nsamples = 16178\nvalue = [11949, 1109, 3120]'),
 Text(52.3125, 67.94999999999993, '\n  (...)  \n'),
 Text(156.9375, 67.94999999999993, '\n  (...)  \n'),
 Text(313.875, 203.8499999999999, 'X[56] <= -0.042\ngini = 0.54\nsamples = 11297\nvalue = [6670, 992, 3635]'),
 Text(261.5625, 67.94999999999993, '\n  (...)  \n'),
 Text(366.1875, 67.94999999999993, '\n  (...)  \n'),
 Text(627.75, 339.74999999999994, 'X[56] <= -0.529\ngini = 0.603\nsamples = 3339\nvalue = [1416, 428, 1495]'),
 Text(523.125, 203.84999

In [ ]:
model_dt_or = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    DecisionTreeClassifier(random_state= 42, max_depth=4)
)

model_dt_or.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['basin', 'region', 'public_meeting',
                                     'scheme_management', 'permit',
                                     'extraction_type', 'extraction_type_group',
                                     'extraction_type_class', 'management',
                                     'management_group', 'payment',
                                     'payment_type', 'water_quality',
                                     'quality_group', 'quantity', 'source',
                                     'source_type', 'source_class',
                                     'waterpoint_type',...
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=4,
                                        

# V. Check Metrics

In [ ]:
print('model_lr accuracy score for training', accuracy_score(y_train, model_lr.predict(X_train)))
print('model_lr accuracy score for val', accuracy_score(y_val, model_lr.predict(X_val)))

model_lr accuracy score for training 0.7386363636363636
model_lr accuracy score for val 0.7376893939393939


In [ ]:
print('model_dt accuracy score for training', accuracy_score(y_train, model_dt.predict(X_train)))
print('model_dt accuracy score for val', accuracy_score(y_val, model_dt.predict(X_val)))

model_dt accuracy score for training 0.8380155723905723
model_dt accuracy score for val 0.7668350168350169


In [ ]:
print('model_dt accuracy score for training', accuracy_score(y_train, model_dt_or.predict(X_train)))
print('model_dt accuracy score for val', accuracy_score(y_val, model_dt_or.predict(X_val)))

model_dt accuracy score for training 0.7084648569023569
model_dt accuracy score for val 0.7024410774410774


# VI. Tune Model

Let's look at the hyperparameter `max_depth`

In [ ]:
depths = range(12, 40, 2)

list(depths)

[12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]

In [ ]:
train_acc = []
val_acc = []

for depth in depths:
   
  tree_model = make_pipeline(
      OrdinalEncoder(),
      SimpleImputer(),
      DecisionTreeClassifier(max_depth=depth, random_state=42)
  )

  tree_model.fit(X_train, y_train)
  train_acc.append(tree_model.score(X_train, y_train))
  val_acc.append(tree_model.score(X_val, y_val))

In [ ]:
val_acc

[0.7608375420875421,
 0.7646254208754208,
 0.7689393939393939,
 0.7669402356902357,
 0.7593644781144782,
 0.7541035353535354,
 0.7523148148148148,
 0.7501052188552189,
 0.7480008417508418,
 0.7494739057239057,
 0.7472643097643098,
 0.749263468013468,
 0.7489478114478114,
 0.7457912457912458]

In [ ]:
plt.plot(depths, train_acc, color='blue', label='training')
plt.plot(depths, val_acc, color='orange', label='validation')
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.legend()
plt.show()

# VII. Communicate Results


### Gini importance

In [ ]:
features = model_dt.named_steps['onehotencoder'].get_feature_names()
gini = model_dt.named_steps['decisiontreeclassifier'].feature_importances_
pd.Series(data=gini, index=features).sort_values(key=abs).tail(10).plot(kind='barh')
plt.ylabel('features')
plt.xlabel('gini importance')

Text(0.5, 111.5, 'gini importance')

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission.head()

,id,status_group
0,37098,functional
1,14530,functional
2,62607,functional
3,46053,functional
4,47083,functional


In [ ]:
# generate CSV
sample_submission.to_csv('sample_submission_Vignesh.csv', index=False)

In [ ]:
# download the file
from google.colab import files
files.download("sample_submission_Vignesh.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
X_test = pd.read_csv('test_features.csv')



In [ ]:
X_test = wrangle(X_test)

X_test.head()



,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
37098,0.0,2012-10-09,Rural Water Supply And Sanitat,0,DWE,31.985658,-3.596360,Kasela,0,Lake Tanganyika,Bufanka Centre,Shinyanga,17,5,Bukombe,Iyogela,0,True,GeoData Consultants Ltd,WUG,NaN,True,0,other,other,other,wug,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
14530,0.0,2012-11-03,Halmashauri Ya Manispa Tabora,0,Halmashauri ya manispa tabora,32.832815,-4.944937,Mbugani,0,Lake Tanganyika,Maendeleo,Tabora,14,6,Tabora Urban,Uyui,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,milky,milky,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump
62607,10.0,2013-02-25,Siter Fransis,1675,DWE,35.488289,-4.242048,Kwa Leosi,0,Internal,Qatabradiki,Manyara,21,1,Babati,Dareda,148,True,GeoData Consultants Ltd,Water Board,NaN,True,2008,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
46053,0.0,2011-08-13,Kkkt,0,KKKT,33.140828,-9.059386,Jangi,0,Lake Rukwa,Chawama,Mbeya,12,6,Mbozi,Iyula,0,False,GeoData Consultants Ltd,VWC,NaN,False,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,seasonal,seasonal,shallow well,shallow well,groundwater,hand pump,hand pump
47083,50.0,2013-02-08,Wateraid,1109,SEMA,34.217077,-4.430529,Mkima,0,Internal,Mkima,Singida,13,1,Iramba,Mtoa,235,True,GeoData Consultants Ltd,WUA,Tyeme water supply,True,2011,mono,mono,motorpump,wua,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe


In [ ]:
(X_test.columns)

39

In [ ]:
(X_train.columns)

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'basin', 'region',
       'region_code', 'district_code', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [ ]:
set(X_test.columns) - set(X_train.columns)

{'date_recorded',
 'funder',
 'installer',
 'lga',
 'num_private',
 'quantity_group',
 'recorded_by',
 'scheme_name',
 'subvillage',
 'ward',
 'wpt_name'}

In [ ]:
def Diff(li1, li2):
#     return list(set(li1) - set(li2)) + list(set(li2) - set(li1))
     return list(set(li1) - set(li2))


cols_to_drop = Diff(X_test.columns, X_train.columns)

cols_to_drop

['wpt_name',
 'ward',
 'date_recorded',
 'num_private',
 'installer',
 'lga',
 'recorded_by',
 'quantity_group',
 'scheme_name',
 'subvillage',
 'funder']

In [ ]:
X_test = X_test.drop(columns=cols_to_drop)

In [ ]:
predictions = pd.DataFrame(data=model_lr.predict(X_test), index=X_test.index)

predictions.columns = ['status_group']

predictions.insert(0, 'id', predictions.index)

predictions = predictions.reset_index(drop=True)


In [ ]:
# generate CSV
predictions.to_csv('sample_submission_Vignesh_class.csv', index=False)

# download
from google.colab import files
files.download("sample_submission_Vignesh_class.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
